In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trainingArr = pd.read_csv('./train-1.csv')
testArr = pd.read_csv('./test.csv')
#getting the y_row of the testArr
x_test = testArr.drop('arousal', axis = 1)
y_test = testArr['arousal']

# x_test = x_test.values.tolist()
# y_test = y_test.values.tolist()

# seperating x and y values accordingly and initializing values needed later
X_train = trainingArr.drop('arousal', axis=1)
y_train = trainingArr['arousal']

print(x_train)